<a href="https://colab.research.google.com/github/InowaR/colab/blob/main/simulate_collisions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import numpy as np
from tqdm import tqdm

def simulate_collisions():
    s, f, n = 10000, 10, 1000000

    np.random.seed(42)
    p = np.random.randint(0, s, (n, 2))
    v = np.random.randint(-5, 6, (n, 2))

    collision_count = 0

    for i in tqdm(range(f)):
        c = np.clip(p + i * v, 0, s-1)
        idx = np.lexsort((c[:, 1], c[:, 0]))
        sorted_c = c[idx]
        diffs = np.diff(sorted_c, axis=0)
        collision_count += np.count_nonzero((diffs == 0).all(axis=1))

    print(f"\nЧастиц: {n:,}, Коллизии: {collision_count}")
    return collision_count

import time
start = time.time()
result = simulate_collisions()
print(f"Время: {time.time() - start:.2f}с")

100%|██████████| 10/10 [00:04<00:00,  2.30it/s]


Частиц: 1,000,000, Коллизии: 50540
Время: 4.41с
